In [1]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
import spacy

In [2]:
# load tanda roberta model
tokenizer = RobertaTokenizer.from_pretrained('models/tanda_roberta_large_asnq_wikiqa/ckpt/')
model = RobertaForSequenceClassification.from_pretrained('models/tanda_roberta_large_asnq_wikiqa/ckpt/') 
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1

# load spacy english language model
nlp = spacy.load('en_core_web_lg')

# load csv
df = pd.read_csv('ds-qa.csv')

In [9]:
nb_rows = df.shape[0]

tanda_loss = [0.0 for _ in range(nb_rows)]
tanda_answer_sentence = ['' for _ in range(nb_rows)]
tanda_sentence_loss = [0.0 for _ in range(nb_rows)]
tanda_sentence_proba = [0.0 for _ in range(nb_rows)]

def getLossAndProbas(q, a, model, labels):
    inputs = tokenizer(q + '? ' + a, return_tensors="pt")
    outputs = model(**inputs, labels=labels)
    loss, logits = outputs[:2]
    return loss.detach().numpy(), torch.sigmoid(logits).detach().numpy()

def divideToSentences(doc, nlp):
    doc = nlp(doc)
    sentences = [sent.string.strip() for sent in doc.sents]
    return sentences

In [12]:
for i, r in df.iterrows():
    if i > 100:
        continue
    print('Working on Question: {} - qid: {} - aid: {}'.format(r['question'], r['qid'], r['aid']))
    loss = 10 # default loss value
    try:
        loss, _ = getLossAndProbas(r['question'], r['answer'], model, labels)
    except:
        print('too long paragraph')
    tanda_loss[i] = loss
    sens = divideToSentences(r['answer'], nlp)
    mnsen, mn, probas = '', 10, [1, 0]
    for sen in sens:
        ls, ps = getLossAndProbas(r['question'], sen, model, labels)
        if ls < mn:
            mn = ls
            probas = ps
            mnsen = sen
    tanda_answer_sentence[i] = mnsen
    tanda_sentence_loss[i] = mn 
    tanda_sentence_proba[i] = 1-probas[0]

df['tanda_loss'] = tanda_loss
df['tanda_answer_sentence'] = tanda_answer_sentence
df['tanda_sentence_loss'] = tanda_sentence_loss
df['tanda_sentence_proba'] = tanda_sentence_proba

df.to_csv('tanda-results.csv')


Working on Question: c# this in method parameter - qid: 29080 - aid: 50380
Working on Question: c# this in method parameter - qid: 29080 - aid: 50381
Working on Question: c# this in method parameter - qid: 29080 - aid: 50382
Working on Question: c# this in method parameter - qid: 29080 - aid: 50383
Working on Question: c# this in method parameter - qid: 29080 - aid: 50384
Working on Question: c# this in method parameter - qid: 29080 - aid: 50385
Working on Question: c# this in method parameter - qid: 29080 - aid: 50386
Working on Question: c# this in method parameter - qid: 29080 - aid: 50387
Working on Question: c# this in method parameter - qid: 29080 - aid: 50388
Working on Question: c# this in method parameter - qid: 29080 - aid: 50389
Working on Question: cluster plot in r - qid: 29081 - aid: 50390
Working on Question: cluster plot in r - qid: 29081 - aid: 50391
Working on Question: cluster plot in r - qid: 29081 - aid: 50392
Working on Question: cluster plot in r - qid: 29081 - a